In [9]:
import jax
# Global flag to set a specific platform, must be used at startup.
#jax.config.update('jax_platform_name', 'cpu')

import os

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='1'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR']='platform'


In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from jax_unirep import get_reps
  
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params

In [2]:
import gc
import jax 
def clear_jax_caches():
  """Utility to clear all the function caches in jax."""
  # main jit/pmap lu wrapped function caches - have to grab from closures
  jax.xla._xla_callable.__closure__[1].cell_contents.clear()
  jax.pxla.parallel_callable.__closure__[1].cell_contents.clear()
  # primitive callable caches
  jax.xla.xla_primitive_callable.cache_clear()
  jax.xla.primitive_computation.cache_clear()
  # jaxpr caches for control flow and reductions
  jax.lax.lax_control_flow._initial_style_jaxpr.cache_clear()
  jax.lax.lax_control_flow._fori_body_fun.cache_clear()
  jax.lax.lax._reduction_jaxpr.cache_clear()
  # these are trivial and only included for completeness sake
  jax.lax.lax.broadcast_shapes.cache_clear()
  jax.xla.xb.get_backend.cache_clear()
  jax.xla.xb.dtype_to_etype.cache_clear()
  jax.xla.xb.supported_numpy_dtypes.cache_clear()
    
def reset_device_memory(delete_objs=True):
    """Free all tracked DeviceArray memory and delete objects.

  Args:
    delete_objs: bool: whether to delete all live DeviceValues or just free.

  Returns:
    number of DeviceArrays that were manually freed.
  """
    dvals = (x for x in gc.get_objects() if isinstance(x, jax.xla.DeviceArray))
    n_deleted = 0
    for dv in dvals:
    
        if not isinstance(dv, jax.xla.DeviceConstant):
            try: 
                dv._check_if_deleted()  # pylint: disable=protected-access
                dv.device_buffer.delete()
                n_deleted += 1
            except:
                pass
        if delete_objs:

            del dv
    del dvals
    gc.collect()
    return n_deleted

In [3]:
#db_path="AMPS_NonAMPs.ready.csv"
db_path="/mnt/vdb/thesis/AMP_NonAMPs.V5_C08_sim60.csv"

In [4]:
df=pd.read_csv(db_path,sep=',',header=0,quoting=csv.QUOTE_ALL)
df # Class 0= AMPs , 1=NonAMps

,ID,Sequence,length,class
0,UPI0006248D48,MTQNVKMGYIRFVVVGMIPWDGQTRTGARAARGQGQGGILKLGAYS...,138,0
1,UPI000195AE9F,MAEEAFDLWNECAKACVLDLKDGVRSSRMSVDPAIADTNGQGVLHY...,460,0
2,UPI000195AE9E,MAEEAFDLWNECAKACVLDLKDGVRSSRMSVDPAIADTNGQGVLHY...,539,0
3,UPI00085D47CF,PLIYLRLLRGQFAGGLRCMCIKWWSGKHPK,30,0
4,UPI00085E5E6F,ARKKAAKAARKKAAKAGGLRCMCIKWWSGKHPK,33,0
...,...,...,...,...
137384,UniRef50_U1L934,MADKQQVDFIRLPSGHRRYLATSIDSFTNAPAEVTR,36,1
137385,UniRef50_A0A6C2WUR6,MPITKHEIPLLECDSNPSAVIIPTHEGLQLSVSMLS,36,1
137386,UniRef50_A0A0H6DSM0,MGYPSMAAALHAAALNIALNIQLNISMRAMLLAFLE,36,1
137387,UniRef50_D3EJ07,MYRKQEKKLAILCFIVAAIMLYGAVKGYLRYMHHFG,36,1


In [5]:
# Setting my chunk size
chunk_size = 500
# Assigning chunk numbers to rows
df['chunk'] = df.index.map(lambda x: int(int(x)/ chunk_size))
# We don't want the 'chunk' and 'index' columns in the output
cols = [col for col in df.columns if col not in ['chunk']]
# groupby chunk and export each chunk to a different csv.
i = 0
for _, chunk in df.groupby('chunk'):
    chunk[cols].to_csv(f'/mnt/vdb/thesis/jax/chunk{i}.csv',sep=",", quotechar='"',index=False, quoting=csv.QUOTE_ALL) # <<-- change this line 
    i += 1
print("complete")

complete


In [6]:
def createREPs(df, filename):
    _h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
    df.drop(columns=['Sequence'], inplace=True)
    df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
    df.to_pickle(filename) # dont forget to change the file name to 0_613834

In [7]:
import glob
appended_reps = []
for infile in glob.glob("/mnt/vdb/thesis/jax/*.pkl"): # <<-- change this line 
    # print(infile)
    appended_reps.append(infile)
print(len(appended_reps))

0


In [ ]:
for infile in glob.glob("/mnt/vdb/thesis/jax/*.csv"): # <<-- change this line 
    #print("Read:"+infile)
    file_name = os.path.basename(infile)
    result="/mnt/vdb/thesis/jax/"+file_name.replace("csv", "pkl") # <<-- change this line 
    if result in appended_reps :
        # print("found then skip : " , result)
        continue
    else:
        df = pd.read_csv(infile) 
        createREPs(df,result)
        print("Save:"+result)
        reset_device_memory()
        clear_jax_caches()
print("complete")

Save:/mnt/vdb/thesis/jax/chunk124.pkl
Save:/mnt/vdb/thesis/jax/chunk50.pkl
Save:/mnt/vdb/thesis/jax/chunk155.pkl
Save:/mnt/vdb/thesis/jax/chunk214.pkl
Save:/mnt/vdb/thesis/jax/chunk10.pkl
Save:/mnt/vdb/thesis/jax/chunk251.pkl
Save:/mnt/vdb/thesis/jax/chunk171.pkl
Save:/mnt/vdb/thesis/jax/chunk130.pkl
Save:/mnt/vdb/thesis/jax/chunk147.pkl
Save:/mnt/vdb/thesis/jax/chunk135.pkl
Save:/mnt/vdb/thesis/jax/chunk245.pkl
Save:/mnt/vdb/thesis/jax/chunk73.pkl
Save:/mnt/vdb/thesis/jax/chunk123.pkl
Save:/mnt/vdb/thesis/jax/chunk133.pkl
Save:/mnt/vdb/thesis/jax/chunk49.pkl
Save:/mnt/vdb/thesis/jax/chunk168.pkl
Save:/mnt/vdb/thesis/jax/chunk175.pkl
Save:/mnt/vdb/thesis/jax/chunk17.pkl
Save:/mnt/vdb/thesis/jax/chunk150.pkl
Save:/mnt/vdb/thesis/jax/chunk264.pkl
Save:/mnt/vdb/thesis/jax/chunk151.pkl
Save:/mnt/vdb/thesis/jax/chunk102.pkl
Save:/mnt/vdb/thesis/jax/chunk36.pkl
Save:/mnt/vdb/thesis/jax/chunk131.pkl
Save:/mnt/vdb/thesis/jax/chunk200.pkl
Save:/mnt/vdb/thesis/jax/chunk97.pkl
Save:/mnt/vdb/thesi

In [11]:
reset_device_memory()
clear_jax_caches()

In [12]:
def mergeDF(path,to_dir,file_name):
    appended_data = []
    for infile in glob.glob(path):
        #print(infile)
        data = pd.read_pickle(infile)
        # store DataFrame in list
        appended_data.append(data)
    result_path=to_dir+"/"+file_name
    print("Save:",result_path)
    appended_data = pd.concat(appended_data)
    appended_data.sort_values(by=['ID'], inplace=True)
    appended_data.to_pickle(result_path)
    return appended_data

In [13]:
# write result
final_df = mergeDF("/mnt/vdb/thesis/jax/*.pkl","/mnt/vdb/thesis/jax","AMPNonAMP.V5_C08_sim60.reps") # <<-- change this line 
final_df

Save: /mnt/vdb/thesis/jax/AMPNonAMP.V5_C08_sim60.reps


,ID,length,class,reps
210,0_antitbpred|antitbpred,33,0,"[0.020596183836460114, 0.05145370587706566, 0...."
371,0_peptideDB.anti|peptideDB.anti,148,0,"[0.006680206395685673, -0.09558607637882233, 0..."
420,1000_pos_train_ds3|pos_train_ds3,86,0,"[0.010705526918172836, 0.00504455529153347, 0...."
168,10023_dbaasp|dbaasp_peptides,36,0,"[0.002072900300845504, -0.07690200954675674, -..."
344,"1003,1011,1019,1027,1035|CancerPPD_l_natural",20,0,"[0.02989775314927101, -0.004465700127184391, -..."
...,...,...,...,...
471,tagenome__1003787_1003787.scaffolds.fasta_scaf...,57,0,"[0.013467248529195786, -0.135259211063385, 0.0..."
447,tagenome__1003787_1003787.scaffolds.fasta_scaf...,47,0,"[0.01930670626461506, -0.030069854110479355, -..."
5,tagenome__1003787_1003787.scaffolds.fasta_scaf...,60,0,"[0.008607540279626846, -0.05736019089818001, 0..."
477,tagenome__1003787_1003787.scaffolds.fasta_scaf...,58,0,"[0.014597401954233646, -0.13994131982326508, 0..."


In [14]:
final_df[final_df["class"] == 1]

,ID,length,class,reps
84,UniRef50_A0A009F8Z1,130,1,"[0.006538190878927708, -0.03369353711605072, 0..."
257,UniRef50_A0A009FV03,37,1,"[0.015707001090049744, -0.05971147492527962, 0..."
285,UniRef50_A0A009GB73,239,1,"[0.009423937648534775, 0.0019337327685207129, ..."
289,UniRef50_A0A009I9E9,99,1,"[0.007063737139105797, -0.0181045513600111, 0...."
402,UniRef50_A0A009IQJ3,210,1,"[9.893722017295659e-05, -0.02460651658475399, ..."
...,...,...,...,...
300,UniRef50_Z9JK72,160,1,"[0.010461339727044106, 0.01646239496767521, 0...."
227,UniRef50_Z9JTC3,1147,1,"[0.011528722010552883, -0.09852015227079391, 0..."
22,UniRef50_Z9JUS5,131,1,"[0.007241042796522379, -0.028920277953147888, ..."
25,UniRef50_Z9JXB0,78,1,"[0.012649418786168098, -0.024078544229269028, ..."


Remove Duplicate

## Split train and test set

In [ ]:
# convert array value from  single column into separate column
df = pd.concat([final_df.pop('reps').apply(pd.Series), final_df['class'],final_df['ID'],final_df['length']], axis=1)
df
#df =final_df[["reps","class"]]
#df
#df_new = df.reps.apply(pd.Series).astype(np.float64)
#df_new['class'] = df['class']
#df_new

In [12]:
X= np.array(final_df['reps'].to_list())
y= np.array(final_df['class'].to_list())

In [13]:
X.shape

(254036, 1900)

In [15]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

# Implement Toy Model (RF)

In [16]:
from sklearn.ensemble import RandomForestClassifier 

classifier = RandomForestClassifier ( random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import plot_confusion_matrix

class_names = ['AMPs', 'NonAMPs']

disp = plot_confusion_matrix(classifier, X_test, y_test,
                            display_labels = class_names,
                            cmap=plt.cm.Blues, xticks_rotation='vertical')

disp.ax_.set_title(" Confusion Matrix")

print(disp.confusion_matrix)
plt.grid(False)
plt.show()